In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import statsmodels.formula.api as smf

In [101]:
file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\all codes\TA_PA_DL_no-prot_somalia.csv"
df = pd.read_csv(file)

In [102]:
#Create the dummy variables
#one for each country
df_dummies = pd.get_dummies(df['admin1'])
df_with_dummies = df.join(df_dummies)

#one for each month
df['month'] = pd.DatetimeIndex(df['time']).month_name()
df_dummies_m = pd.get_dummies(df['month'])
df_with_dummies = df_with_dummies.join(df_dummies_m)

#one for each for each country-month pair
df_dummies_mr = pd.get_dummies(df['admin1'] + df['month'])
df_with_dummies = df_with_dummies.join(df_dummies_mr)
df_with_dummies = df_with_dummies.replace({True: 1, False: 0})

#df_with_dummies = df_with_dummies.drop(['month'], axis=1)

In [103]:
len(df_with_dummies)

5616

In [104]:
#reschpe the dataframe only taking the rows within a time frame
#df_with_dummies = df_with_dummies[df_with_dummies['time'] >= '2000-01']
#df_with_dummies = df_with_dummies[df_with_dummies['time'] <= '2019-12']

In [105]:
y_var_name = 'conflicts'
X_var_names = ['TA','PA','DL']

In [106]:
# Regression expression for OLS with dummies

unit_names = df['admin1'].unique().tolist()
unit_names.sort()
unit_names_t = df['month'].unique().tolist()
unit_names_mr = (df['admin1'] + df['month']).unique().tolist()

lsdv_expr = y_var_name + ' ~ '
i = 0
for X_var_name in X_var_names:
    if i > 0:
        lsdv_expr = lsdv_expr + ' + ' + X_var_name
    else:
        lsdv_expr = lsdv_expr + X_var_name
    i = i + 1
for dummy_name in unit_names[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name
for dummy_name_t in unit_names_t[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name_t
for dummy_name_mr in unit_names_mr[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name_mr
 
print('Regression expression for OLS with dummies=' + lsdv_expr)

Regression expression for OLS with dummies=conflicts ~ TA + PA + DL + Awdal + Bakool + Banadir + Bari + Bay + Galgaduud + Gedo + Hiraan + Lower_Juba + Lower_Shabelle + Middle_Juba + Middle_Shabelle + Mudug + Nugaal + Sanaag + Sool + Togdheer + January + February + March + April + May + June + July + August + September + October + November + AwdalJanuary + BakoolJanuary + BanadirJanuary + BariJanuary + BayJanuary + GalgaduudJanuary + GedoJanuary + HiraanJanuary + Lower_JubaJanuary + Lower_ShabelleJanuary + Middle_JubaJanuary + Middle_ShabelleJanuary + MudugJanuary + NugaalJanuary + SanaagJanuary + SoolJanuary + TogdheerJanuary + Woqooyi_GalbeedJanuary + AwdalFebruary + BakoolFebruary + BanadirFebruary + BariFebruary + BayFebruary + GalgaduudFebruary + GedoFebruary + HiraanFebruary + Lower_JubaFebruary + Lower_ShabelleFebruary + Middle_JubaFebruary + Middle_ShabelleFebruary + MudugFebruary + NugaalFebruary + SanaagFebruary + SoolFebruary + TogdheerFebruary + Woqooyi_GalbeedFebruary + Awd

In [107]:
lsdv_model = smf.ols(formula=lsdv_expr, data=df_with_dummies)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())
print(len(df_with_dummies))

                            OLS Regression Results                            
Dep. Variable:              conflicts   R-squared:                       0.411
Model:                            OLS   Adj. R-squared:                  0.387
Method:                 Least Squares   F-statistic:                     17.26
Date:                Tue, 25 Jul 2023   Prob (F-statistic):               0.00
Time:                        17:16:50   Log-Likelihood:                -21005.
No. Observations:                5616   AIC:                         4.245e+04
Df Residuals:                    5397   BIC:                         4.390e+04
Df Model:                         218                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [108]:
#export the summary to a txt file
with open('model_results.txt', 'w') as fh:
    fh.write(lsdv_model_results.summary().as_text())

In [ ]:
#shifta i conflitti di n mesi